In [ ]:
from getpass import getpass
import subprocess, sys
ORG = 'mctwork2'  # <-- replace after pushing
REPO = 'tpok003'  # <-- replace after pushing
REF = 'main'  # e.g. 'main' or a tag like 'v0.1.0'
TOKEN = getpass('GitHub token (read-only): ')

subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-U',
                      'pandas==2.2.2', 'openpyxl>=3.1,<4'])
url = f'git+https://{TOKEN}@github.com/{ORG}/{REPO}.git@{REF}'
subprocess.check_call([sys.executable, '-m', 'pip', 'install', url])


In [ ]:
# @title 🖥️ UI: Выберите 2 файла и дату — код выполнит nbutest.py без изменений
!pip -q install gradio>=4.0 openpyxl pandas
import os, json, runpy, shutil
import gradio as gr

# Убедимся, что status2_map.json доступен в рабочей директории
EXAMPLES_DIR = '/content/examples'
os.makedirs(EXAMPLES_DIR, exist_ok=True)
if os.path.exists('/content/colab-merge-project/examples/status2_map.json'):
    shutil.copy('/content/colab-merge-project/examples/status2_map.json', '/content/status2_map.json')
elif os.path.exists('examples/status2_map.json'):
    shutil.copy('examples/status2_map.json', '/content/status2_map.json')

def run_nbutest(file1, file2, date_str):
    if file1 is None or file2 is None:
        raise gr.Error('Загрузите оба файла (CSV/XLSX)')
    if not date_str:
        raise gr.Error('Введите дату в формате ДД.ММ.ГГГГ (например, 21.08.2025)')
    # Пишем временный app_settings.json, как ожидает nbutest.py
    settings = {
        'files': [file1.name, file2.name],
        'поточнадата': date_str
    }
    with open('/content/app_settings.json', 'w', encoding='utf-8') as f:
        json.dump(settings, f, ensure_ascii=False, indent=2)
    # Запускаем nbutest.py (используем runpy, чтобы выполнить как скрипт)
    try:
        runpy.run_path('/content/colab-merge-project/scripts/nbutest.py', run_name='__main__')
    except SystemExit:
        # Некоторые скрипты могут вызвать sys.exit — игнорируем, если файл создан
        pass
    # По коду nbutest.py выходной файл называется result_ДД.ММ.ГГГГ.xlsx (или похожий)
    # Соберём кандидат по дате, а также найдём последний .xlsx созданный скриптом
    candidates = [
        f'/content/result_{date_str}.xlsx',
        f'result_{date_str}.xlsx'
    ]
    for c in candidates:
        if os.path.exists(c):
            return c
    # Фолбэк: ищем свежий .xlsx в /content
    latest = None
    for fname in os.listdir('/content'):
        if fname.lower().endswith('.xlsx') and fname.startswith('result_'):
            p = os.path.join('/content', fname)
            if latest is None or os.path.getmtime(p) > os.path.getmtime(latest):
                latest = p
    if latest:
        return latest
    raise gr.Error('Выходной файл не найден. Проверьте логи выполнения.')

with gr.Blocks() as demo:
    gr.Markdown('''# 📘 Обработка по полной логике nbutest.py
Выберите **два входных файла** и введите **дату** (ДД.ММ.ГГГГ). Скрипт использует вашу же логику из `scripts/nbutest.py` — мы лишь подставляем выбранные файлы и дату во временный `app_settings.json` и запускаем его.
''')
    with gr.Row():
        f1 = gr.File(label='Файл 1 (Excel/CSV)')
        f2 = gr.File(label='Файл 2 (Excel/CSV)')
    date_input = gr.Textbox(label='Дата (ДД.ММ.ГГГГ)', placeholder='например, 21.08.2025')
    go = gr.Button('Запустить обработку')
    out_file = gr.File(label='Результат (XLSX)')
    go.click(run_nbutest, inputs=[f1, f2, date_input], outputs=out_file)

demo.launch()
